## **SQL Skript**

create table transactions as (select transaction_id, 0 as new_product_id, customer_id, transaction_date, online_order, order_status from trans);

create table brands (brand_id serial primary key, brand varchar(30));

insert into brands (brand) select distinct brand from trans;

create table product_lines (product_line_id serial primary key, product_line varchar(30));

insert into product_lines (product_line) select distinct product_line from trans;

create table product_classes (product_class_id serial primary key, product_class varchar(30));
insert into product_classes (product_class) select distinct product_class from trans;

create table product_sizes (product_size_id serial primary key, product_size varchar(30));
insert into product_sizes (product_size) select distinct product_size from trans;

update cust set gender='Female' where gender='F' or gender='Femal'
update cust set gender='Male' where gender='M'

create table genders (gender_id serial primary key, gender varchar(30));
insert into genders (gender) select distinct gender from cust;

create table industries (job_industry_category_id serial primary key, job_industry_category varchar(30));
insert into industries (job_industry_category) select distinct job_industry_category from cust;

create table wealth_segments (wealth_segment_id serial primary key, wealth_segment varchar(30));
insert into wealth_segments (wealth_segment) select distinct wealth_segment from cust;

update cust set state='NSW' where state='New South Wales'
update cust set state='VIC' where state='Victoria'

create table countries (country_id serial primary key, country varchar(30));
insert into countries (country) select distinct country from cust;

create table customers as (select customer_id, first_name, last_name, gender_id, "DOB" as dob, job_title, job_industry_category_id, wealth_segment_id, deceased_indicator, owns_car, address, postcode, state, country_id, property_valuation from cust as c, genders g, industries i, wealth_segments w, countries c1 where c.gender=g.gender and c.job_industry_category=i.job_industry_category and c.wealth_segment=w.wealth_segment and c.country=c1.country);
ALTER TABLE customers ALTER COLUMN customer_id, SET NOT NULL;
ALTER TABLE customers ALTER COLUMN gender_id SET NOT NULL;

create table products(
new_product_id serial primary key,
product_id int4,
brand_id int4 not null,
product_line_id int4 not null,
product_class_id int4 not null,
product_size_id int4 not null,
list_price numeric(8,2),
standard_cost numeric(8,2),
CONSTRAINT fk_brand FOREIGN KEY (brand_id) references brands(brand_id),
CONSTRAINT fk_product_line FOREIGN KEY (product_line_id) references product_lines(product_line_id),
CONSTRAINT fk_product_class FOREIGN KEY (product_class_id) references product_classes(product_class_id),
CONSTRAINT fk_product_size FOREIGN KEY (product_size_id) references product_sizes(product_size_id))

update trans set standard_cost='0,00' where standard_cost=''


insert into products (product_id, brand_id, product_line_id, product_class_id, product_size_id, list_price, standard_cost) select product_id, brand_id, product_line_id, product_class_id, product_size_id, replace(t.list_price, ',', '.')::numeric(8,2) as list_price, replace(t.standard_cost, ',', '.')::numeric(8,2) as standard_cost from trans t, brands b, product_lines pl, product_classes pc, product_sizes ps where t.brand=b.brand and t.product_line=pl.product_line and t.product_class=pc.product_class and t.product_size=ps.product_size group by product_id, brand_id, product_line_id, product_class_id, product_size_id, list_price, standard_cost;

create table order_statuses (order_status_id serial primary key, order_status varchar(30));
insert into order_statuses (order_status) select distinct order_status from trans;

create unique index on customers (customer_id);

CREATE TABLE transactions (
	transaction_id int4 not null,
	new_product_id int4 not null,
	customer_id int4 not null,
	transaction_date date,
	online_order bool,
	order_status_id int4  not null,
CONSTRAINT fk_new_product_id FOREIGN KEY (new_product_id) references products(new_product_id),
CONSTRAINT fk_customer_id FOREIGN KEY (customer_id) references customers(customer_id),
CONSTRAINT fk_order_status_id FOREIGN KEY (order_status_id) references order_statuses(order_status_id));
with aa as (select transaction_id, new_product_id, t.customer_id, to_date(transaction_date, 'dd.mm.yyyy'), online_order, order_status_id from trans t, products p, customers c, order_statuses os, brands b, product_lines pl, product_classes pc, product_sizes ps where t.product_id=p.product_id and t.customer_id=c.customer_id and t.order_status=os.order_status and p.brand_id=b.brand_id and p.product_line_id=pl.product_line_id and p.product_class_id=pc.product_class_id and p.product_size_id=ps.product_size_id and t.brand=b.brand and t.product_line=pl.product_line and t.product_class=pc.product_class and t.product_size=ps.product_size and replace(t.list_price, ',', '.')::numeric(8,2)= p.list_price group by  transaction_id, new_product_id, t.customer_id, transaction_date, online_order, order_status_id)
insert into transactions select * from aa;

create table postcodes (
postcode int4 primary key not null,
state varchar(10) not null,
country_id int4 not null,
CONSTRAINT fk_country_id FOREIGN KEY (country_id) references countries(country_id));
insert into postcodes (select postcode, state, 1 from customers group by postcode, state);

alter table customers drop column state;
alter table customers drop column country_id;

## **Нормализация**

Таблицы transaction и customer, приведенные в формате MS Excel, находятся во второй нормальной форме, так как атрибуты атомарны, отсутствуют дубликаты, а также каждый столбец, который не является ключом, зависит от первичного ключа.

Для того, чтобы привести базу данных к третьей нормальной форме необходимо, чтобы не было столбцов с неключевыми значениями, которые зависят от других неключевых значений.
Для этого необходимо произвести декомпозицию и избавиться от транзитивности имеющихся таблиц. Полученная БД 3-НФ представлена на схеме.
В целях экономии знимаемого места таблицы были разбиты на дополнительные справочники

![picture_of_diag](1p.png)

Эта же архитектура была реализована в DBeaver. Некоторые связи и загрузка исходных файлов (cust, trans) были реализованы без написания скриптов.

Исходные данные импортированы в полном объеме

![picture_of_diag](2p.png)

## **Table trans**

![picture_of_diag](p.png)

## **Table cust**

![picture_of_diag](pp.jpg)